<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 440%; font-weight: 700; text-align: center; color: Plum;">
<br>
寫個無聊「遊戲」來練功
<br>
<br>
<br>
<!-- <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 180%; text-align: center;"> -->

</div>

In [1]:
%%javascript
// 設定output文字顏色。
document.styleSheets[0].addRule('body', 'color: #87CEFA !important;')

<IPython.core.display.Javascript object>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
遊戲規則
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 搜集或隨機產生一個英文單字庫(目前該字庫已有37多萬字)。
* 從字庫中取出每一字檢查是否為`ShiftLex`。
* 所謂`ShiftLex`，是筆者和ChatGPT討論後創出的新詞，意思是：
    * 該字本身是有意義的英文單字。
    * 將每一個字母輪流移至(shift)字首，其餘字母順序不變。所有移動後形成的新字都必須為有效英文單字。
* `ShiftLex`範例(目前暫未找出4個字母以上的`ShiftLex`)：
    * art -> rat -> tar
    * tea -> eta -> ate  
* 夠無聊了吧？依這些功能根本談不上是遊戲，充其量只是練功吧了。倒是可以給比較少寫程式的同學作個step-by-step示範。不過這是急就章下產品，來不及優化，不一定是很好的coding示範。各位請多多給予批評和建議。


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
如何「隨機產生」英文單字？
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 簡單：用[random_word package](https://pypi.org/project/Random-Word/)即可。
* `$ pip install random-word`


In [2]:
from random_word import RandomWords

r = RandomWords()
words = set()
for i in range(9):
    # print(f'{i}', end='\r')
    words.add(r.get_random_word())
print(f'{len(words) =}')
print(f'{words}')

len(words) =9
{'swird', 'catalyzers', 'rudolphine', 'quadricone', 'countercultural', 'surgerize', 'serpentinize', 'amyloplastid', 'almanac'}


<div style="text-align:center"><img src="https://hackmd.io/_uploads/SyI5hFsS3.png" width="600"/></div>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
不過這樣隨機產生太慢了！
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 直接到該[package的github](https://github.com/vaibhavsingh97/random-word)，將它[整個字庫](https://raw.githubusercontent.com/vaibhavsingh97/random-word/master/random_word/database/words.json)copy出來比較快。
* 不過這個字庫的字偏長且冷僻，筆者又從其他提供單字的網站補充了很幾千個較短的常用單字，經過簡單整理後存入`words_raw.dat`檔。
* 這個檔案尚待進一步去蕪存菁，例如過濾重複、刪除過短的字、刪除帶數字或特殊符號者...等等。
* 整理的code在下面：

In [16]:
# 單字字庫整理-1：刪除重複及全轉為小寫
with open('./words_raw.dat', 'r') as f1:
    words = list(set(f1.read().lower().split('\n')))

# 單字字庫整理-2：最少3個字母、字母沒有完全相同、且沒有數字和特殊符號
# 者才納入最終字庫。
symbols = ('.', ',', '?', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '<', '>', '"', "'", '|', ':', ';', '~', '!', '`', '@', '#', '$', '%', '^', '&', '*', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '=', 'þ')
with open('./words_final.dat', 'w') as f2:
    for index, word in enumerate(words):
        leng_word = len(word)
        if leng_word >= 3 and word != word[0]*leng_word and not [char for char in word if char in symbols]:
            print(f'{index}', end='\r')
            _ = f2.write(f'{word}\n')
            f2.flush()

<div style="text-align:center"><img src="https://hackmd.io/_uploads/HJsUHmqEn.jpg" width="600"/></div>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
資料整理好後，正式幹活...
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 目前面臨的問題是：如何得知某個字母的組合是「有效的英文單字」？ChatGPT API等AI工具一定可以搞定，不過殺雞好像不必用牛刀，而且得付token費。暫不考慮。
* 稍為搜尋一下，找到以下3套Python的模組，大扺可滿足需求：
    1. enchant
    2. nltk
    3. spellchecker
* 也許還有別的更好用模組，待日後很有空時再找好了。目前先試用這3套，看效果如何。

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
1. enchant
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install pyenchant`
* [PyEnchant Introdution](https://pyenchant.github.io/pyenchant/)

In [7]:
import enchant

# 使用前
enchant.list_languages()

['en', 'en_AU', 'en_CA', 'en_GB', 'en_US', 'en_ZA']

<div style="text-align:center"><img src="https://hackmd.io/_uploads/r1CGkKF1p.png" width="600"/></div>

In [15]:
import enchant

spellings = ('en_US', 'en_')   # 'en_'好像是英美spelling通吃。
for spelling in spellings:
    d = enchant.Dict(spelling)   # 先設定使用哪種「英文」
    word1 = 'center'   # American spelling
    word2 = '??????'   # British spelling
    print(f"{spelling}\t{word1}: {d.check(word1)}\t{word2}: {d.check(word2)}")

en_US	center: True	??????: False
en_	center: True	??????: False


In [18]:
with open('./words_final.dat', 'r') as f:
    words = f.read().split('\n')

words[:5]
words[-5:]

['balsamodendron', 'undershorts', 'stoures', 'perfecter', 'unnooked']

['sporogenesis', 'tetanilla', 'cismontane', 'breastpiece', 'geostrategic']

In [249]:
# v1: 直白寫
import enchant

d = enchant.Dict('en_')
shift_lexes = []
for word in words:
    fitting = True
    for index, alphabet in enumerate(word):
        print(f'{index:,} / {len(words):,}', end='\r')
        this_rotation = f'{alphabet}{word[:index]}{word[index+1:]}'
        if not d.check(this_rotation.lower()):
            fitting = False
            break
    if fitting:
       shift_lexes.append(word)


print(shift_lexes)
print(len(shift_lexes))

['tea', 'ups', 'alp', 'rob', 'arm', 'apt', 'int', 'add', 'asp', 'amt', 'opp', 'app', 'own', 'art', 'wee', 'eel', 'opt', '']
18


In [19]:
# v2: 改為function，同時加上「最少次數」，讓較多的字符合標準。
import enchant
def is_valid_shift_lex(lex: str, rotations: int) -> bool:
    d = enchant.Dict('en_')
    fittings = 0
    for index, alphebet in enumerate(lex):
        this_rotation = f'{alphebet}{lex[:index]}{lex[index+1:]}'
        if d.check(this_rotation):
            fittings += 1
            if fittings >= rotations:
                break
    return fittings >= rotations or fittings == len(lex)

shift_lexes = []
for index, word in enumerate(words):
    print(f'{index:,} / {len(words):,}', end='\r')
    if is_valid_shift_lex(lex=word, rotations=3):
       shift_lexes.append(word)

print(shift_lexes)
print(len(shift_lexes))

['arts', 'huts', 'laps', 'adds', 'earring', 'wees', 'appal', 'ribbed', 'assess', 'opts', 'doors', 'opt', 'lumps', 'ones', 'pass', 'arms', 'parses', 'asps', 'hits', 'opp', 'mites', 'arcing', 'ups', 'cares', 'veneer', 'labs', 'avers', 'arm', 'watt', 'caress', 'ribbers', 'manatee', 'educes', 'abbes', 'eels', 'apps', 'orts', 'apt', 'eel', 'ribber', 'manatees', 'robs', 'oops', 'isms', 'acme', 'asp', 'steer', 'hanks', 'eviler', 'greet', 'angers', 'door', 'windless', 'rapt', 'int', 'eats', 'rob', 'ends', 'wee', 'app', 'steers', 'east', 'acres', 'acre', 'greets', 'alps', 'alp', 'ides', 'ores', 'pots', 'watts', 'own', 'weer', 'amt', 'aids', 'tea', 'ribbing', 'kiss', 'art', 'wast', 'hist', 'hears', 'lams', 'veer', 'add', 'cats', 'arrest', 'lats', 'arced', 'abbe', 'alts', 'hips']
92


In [23]:
# v2: 改為function，同時加上「最少次數」，讓較多的字符合標準。
import enchant
def is_valid_shift_lex(lex: str, rotations: int) -> bool:
    d = enchant.Dict('en_')
    fittings = 0
    for index, alphebet in enumerate(lex):
        this_rotation = f'{alphebet}{lex[:index]}{lex[index+1:]}'
        if d.check(this_rotation):
            print(this_rotation)
            fittings += 1
            if fittings >= rotations:
                break
    return fittings >= rotations or fittings == len(lex)

word = 'acres'
_ = is_valid_shift_lex(lex=word, rotations=3)


acres
cares
races


In [29]:
# v3: 改為class
import enchant

class ShiftLex():
    def __init__(self):
        self.d = enchant.Dict('en_')

    def is_valid(self, lex: str, rotations: int) -> bool:
        d = enchant.Dict('en_')
        fittings = 0
        for index, alphebet in enumerate(lex):
            this_rotation = f'{alphebet}{lex[:index]}{lex[index+1:]}'
            if d.check(this_rotation):
                fittings += 1
                if fittings >= rotations:
                    break
        return fittings >= rotations or fittings == len(lex)


lex = ShiftLex()
shift_lexes = []
for index, word in enumerate(words):
    print(f'{index:,} / {len(words):,}', end='\r')
    if lex.is_valid(lex=word, rotations=3):
       shift_lexes.append(word)

print(shift_lexes)
print(len(shift_lexes))

['arts', 'huts', 'laps', 'adds', 'earring', 'wees', 'appal', 'ribbed', 'assess', 'opts', 'doors', 'opt', 'lumps', 'ones', 'pass', 'arms', 'parses', 'asps', 'hits', 'opp', 'mites', 'arcing', 'ups', 'cares', 'veneer', 'labs', 'avers', 'arm', 'watt', 'caress', 'ribbers', 'manatee', 'educes', 'abbes', 'eels', 'apps', 'orts', 'apt', 'eel', 'ribber', 'manatees', 'robs', 'oops', 'isms', 'acme', 'asp', 'steer', 'hanks', 'eviler', 'greet', 'angers', 'door', 'windless', 'rapt', 'int', 'eats', 'rob', 'ends', 'wee', 'app', 'steers', 'east', 'acres', 'acre', 'greets', 'alps', 'alp', 'ides', 'ores', 'pots', 'watts', 'own', 'weer', 'amt', 'aids', 'tea', 'ribbing', 'kiss', 'art', 'wast', 'hist', 'hears', 'lams', 'veer', 'add', 'cats', 'arrest', 'lats', 'arced', 'abbe', 'alts', 'hips']
92


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
2. nltk(Natural Language Toolkit)
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install nltk`
* [nltk documentation](https://www.nltk.org/)

In [95]:
from nltk.corpus import words

print(f"{'language' in words.words() = }")
print(f"{'langage' in words.words() = }")

'language' in words.words() = True
'langage' in words.words() = False


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
3. spellchecker
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `$ pip install pyspellchecker`
* [spellcheckertk documentation](https://pyspellchecker.readthedocs.io/en/latest/)

In [97]:
from spellchecker import SpellChecker

spell = SpellChecker()

print(f"{'language'}: {'language' == spell.correction('language')}")
print(f"{'langage'}: {'langage' == spell.correction('langage')}")

language: True
langage: False


<div style="text-align:center"><img src="https://i.imgur.com/ho0NZ9y.png" width="600"/></div>

In [85]:
lexicons = [('Alex', 'alex'), ('Taipei', 'taipei'), ('Europe', 'europe'), ('Microsoft', 'microsoft'), ('Linux', 'linux'), ('NATO', 'nato'), ('Christmas', 'christmas')]

In [86]:
import enchant

d = enchant.Dict('en_')   # 先設定使用哪種「英文」
for lex in lexicons:
    print(f"{lex[0]}: {d.check(lex[0])}\t\t{lex[1]}: {d.check(lex[1])}")

Alex: True		alex: False
Taipei: True		taipei: False
Europe: True		europe: False
Microsoft: True		microsoft: False
Linux: True		linux: False
NATO: True		nato: False
Christmas: True		christmas: False


In [91]:
from nltk.corpus import words

for lex in lexicons:
    print(f"{lex[0]}: {lex[0] in words.words()}\t\t{lex[1]}: {lex[0] in words.words()}")

Alex: True		alex: True
Taipei: False		taipei: False
Europe: False		europe: False
Microsoft: False		microsoft: False
Linux: False		linux: False
NATO: False		nato: False
Christmas: True		christmas: True


In [92]:
from spellchecker import SpellChecker

spell = SpellChecker()
for lex in lexicons:
    print(f"{lex[0]}: {lex[0] == spell.correction(lex[0])}\t\t{lex[1]}: {lex[1] == spell.correction(lex[1])}")

Alex: True		alex: True
Taipei: True		taipei: True
Europe: True		europe: True
Microsoft: False		microsoft: False
Linux: False		linux: False
NATO: True		nato: True
Christmas: True		christmas: True


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
結論
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 本「遊戲」非常無趣，不過可借此練一下基本功。
* 如果不用ChatGPT等AI方法，要判斷英文單字，暫時找不到一個非常理想的工具。